In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os
from autogen_ext.models.ollama import OllamaChatCompletionClient

In [7]:
api_key = os.getenv("GLM_API_KEY", "your_api_key_here")
base_url = os.getenv("GLM_BASE_URL", "your_api_base_url_here")

In [8]:
(api_key, base_url) 

('c2dbfa218c1f3a8c11432259644c5913.QCQk336aSIsrJ94L',
 'https://open.bigmodel.cn/api/paas/v4/')

In [9]:
# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(

    model="glm-4.5",
    api_key=api_key,
    base_url=base_url,
    model_info={
        'vision': False,
        'function_calling': True,
        'json_output': True,
        'structured_output': False,
        'family': "glm",
    },
)

model_client = OllamaChatCompletionClient(
            model="qwen3:30b",
            model_info={
                'vision': False,
                'function_calling': True,
                'json_output': True,
                'structured_output': False,
                'family': "qwen",
            },
            options={
                'num_ctx': int(os.getenv("NUM_CTX", "6000")),
                'stream': True,  # 开启流式输出
            }
        )


In [10]:
streaming_assistant = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You are a helpful assistant.",
    model_client_stream=True,  # Enable streaming tokens.
 )



In [ ]:
#流式输出streaming_assistant的内容


In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
#model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client, model_client_stream=True)  # Enable streaming tokens.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
async for message in stream: #streaming_assistant.run_stream(task="Name two cities in South America"):  # type: ignore
    print(message.content, end='', flush=True)


In [ ]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.run_stream(task="Find information on AutoGen"):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.run_stream(task="Find information on AutoGen"),
        output_stats=False,  # Enable stats printing.
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()